### Imports

In [88]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time, os
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from scipy.constants import speed_of_light as c
import scipy.io
from collections import OrderedDict
import torch.fft as tfft
import datetime
%matplotlib inline

from torch.utils.data import Dataset, DataLoader
# %load_ext line_profiler

gpu = torch.device("cpu")

# ## Sets everything to double point precision (use with gradcheck)
# torch.set_default_dtype(torch.float64)
import os
os.environ["MKL_THREADING_LAYER"] = "GNU"

In [89]:
# A list of transforms to perform on all image data
# First turn image data into pytorch tensor, then normalize it with mean and std of 0.5
# (only one channel per image) since there's only one element in each tuple
# img_size = (14, 14)
# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.Resize(img_size, interpolation=2), torchvision.transforms.ToTensor(),
#      torchvision.transforms.Normalize((0.5,), (0.5,))])

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.Resize(img_size, interpolation=2), torchvision.transforms.ToTensor()])

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.ToTensor()])

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.ToTensor(),
#      torchvision.transforms.Normalize((0.5,), (0.5,))])

# Custom L2 normalization transformation
class L2Normalize:
    def __call__(self, tensor):
        return tensor / tensor.norm(p=2)

# Define the transformations
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),    # Convert PIL image to Tensor
    torchvision.transforms.Normalize((0.5,), (0.5,))            
])

# Downloading the training MNIST data and applying the above transform
# (The data type is a dataset, iterated by 60000 tuples, where the first element
# of each tuple contains the tensor with the data, and the second element is an integer)
# Note: each image in both the trainset and testset have shape [1, 28, 28]
dataset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

num_training = 50000
num_validation = 10000 # 10000
random_numbers = np.random.choice(np.arange(60000), size=num_training+num_validation, replace=False)

# Convert the NumPy range to a Python list
subset_indices = list(random_numbers)

# Create the subset using torch.utils.data.Subset
subset_dataset = torch.utils.data.Subset(dataset, subset_indices)

trainset, validset = torch.utils.data.random_split(subset_dataset, [num_training, num_validation])

# Downloading the test MNIST data and applying the above transform
# (The data type is a dataset, iterated by 10000 tuples, where the first element
# of each tuple contains the tensor with the data, and the second element is an integer)
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

num_test = 10000 # 10000
random_numbers2 = np.random.choice(np.arange(10000), size=num_test, replace=False)

# Convert the NumPy range to a Python list
subset_indices2 = list(random_numbers2)

# Create the subset using torch.utils.data.Subset
subset_testset = torch.utils.data.Subset(testset, subset_indices2)

# The loaded sets of data, ready to be inputted into a neural net.
# Each "loader" splits all the data into specified batches.  When iterated, there are 
# (num of feature maps / num of batches) items, and each item contains a list of length 2,
# where the first element contains a tensor of (num of batches) elements containing the tensors of data,
# and the second element contains a tensor of (num of batches) elements containing the tensors of labels
batch_size = 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=True, pin_memory=True)
testloader = torch.utils.data.DataLoader(subset_testset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [90]:
num_test = 1000 # 10000
random_numbers2 = np.random.choice(np.arange(10000), size=num_test, replace=False)

# Convert the NumPy range to a Python list
subset_indices2 = list(random_numbers2)

# Create the subset using torch.utils.data.Subset
subset_validset = torch.utils.data.Subset(validset, subset_indices2)

batch_size = 64
validloader1000 = torch.utils.data.DataLoader(subset_validset, batch_size=batch_size, 
                                              shuffle=False, pin_memory=True)

### Custom secure IP network

In [120]:
class SecureMVMnoab_homodyne(nn.Module):
    def __init__(self, weightshape, noise=False, FSD=None):
        super(SecureMVMnoab_homodyne, self).__init__()
        self.weightshape = weightshape
        self.noise = noise
        self.FSD = FSD #First Standard Deviation
        self.W = nn.Parameter(torch.empty(*weightshape))
        nn.init.kaiming_uniform_(self.W, a=0)

    def forward(self, z):
        # Perform the matrix multiplication with alpha
        alpha_W = self.FSD * self.W
        x = z / torch.norm(z, p=2, dim=-1, keepdim=True)
        pre_homodyne_result = torch.matmul(x, alpha_W.t())

        # Calculate the mean of the homodyne result
        measuredhomodynemean = torch.real(pre_homodyne_result)
        
        if self.noise:
            # Add noise
            noise = torch.normal(0., 1., size=measuredhomodynemean.shape, device=measuredhomodynemean.device)
            measuredhomodynemean += noise
        
        return measuredhomodynemean

In [116]:
class SecureNN2layernoab_homodyne(nn.Module):
    
    def __init__(self, inputsize, hiddensize, outputsize, noise=False, FSD1=None,FSD2=None):
        
        super(SecureNN2layernoab_homodyne, self).__init__()

        self.flatten = nn.Flatten()
        self.secure1 = SecureMVMnoab_homodyne([hiddensize, inputsize], noise=noise, FSD=FSD1)
        self.secure2 = SecureMVMnoab_homodyne([outputsize, hiddensize], noise=noise, FSD=FSD2)
    
    def forward(self, x):
        
        x = self.flatten(x) 
        x = self.secure1(x)
        x = torch.relu(x)
#         x = x / torch.norm(x, p=2, dim=-1, keepdim=True)
        x = self.secure2(x)
        
        return x

In [96]:
loaderfortraining = trainloader
loaderforvalid = validloader

### MNIST stuff

In [97]:
# Define the model
class SimpleNN(nn.Module):
    
    def __init__(self, inputsize, hiddensize, outputsize):
        
        super(SimpleNN, self).__init__()

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(inputsize, hiddensize, bias=False)
        self.linear2 = nn.Linear(hiddensize, outputsize, bias=False)
    
    def forward(self, x):
        
        x = self.flatten(x) 
        x = self.linear1(x)
        x = torch.relu(x)
#         x = x / torch.norm(x, p=2, dim=-1, keepdim=True)
        x = self.linear2(x)
#         x = torch.abs(x)
        
        return x

In [99]:
def calculate_accuracy(loader, model, secure=False, printprogress=False):
    
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    
    if secure:
        PDmeansarr = []
        PDrealizedarr = []
    
    with torch.no_grad():
        for images, labels in loader:
            
            if not secure:
                outputs = model(images)
            else:
                outputs, PDmeans, PDrealized = model(images)
                PDmeansarr.append(PDmeans)
                PDrealizedarr.append(PDrealized)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            if printprogress:
                print(f"{total} test batches done")
                
    model.train()  # Set the model back to training mode
    
    if not secure:
        return 100 * correct / total
    else:
        return 100 * correct / total, PDmeansarr, PDrealizedarr  

In [123]:
# Initialize the model
inputsize, hiddensize, outputsize = 784, 784, 10

model = SimpleNN(inputsize, hiddensize, outputsize)
# model = SimpleNN1layer(inputsize, outputsize)

num_epochs = 100
learning_rate = 1e-3

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start = time.time()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loaderfortraining):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#         # Clamping the weights to be within [0, 1]
#         with torch.no_grad():
#             for param in model.parameters():
#                 param.clamp_(-1, 1)
        
    # Calculate training and validation accuracy
    train_accuracy = calculate_accuracy(loaderfortraining, model)
    val_accuracy = calculate_accuracy(loaderforvalid, model)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f},\
    Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%,\
    Time: {(time.time()-start)/(epoch+1)}')
        
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 0.0694,    Training Accuracy: 94.54%, Validation Accuracy: 93.93%,    Time: 26.322789669036865
Epoch [2/100], Loss: 0.0064,    Training Accuracy: 96.42%, Validation Accuracy: 95.95%,    Time: 27.478947043418884
Epoch [3/100], Loss: 0.2905,    Training Accuracy: 97.64%, Validation Accuracy: 96.91%,    Time: 27.79883098602295
Epoch [4/100], Loss: 0.1523,    Training Accuracy: 96.60%, Validation Accuracy: 95.68%,    Time: 27.995770037174225
Epoch [5/100], Loss: 0.0047,    Training Accuracy: 97.96%, Validation Accuracy: 97.06%,    Time: 28.08621163368225
Epoch [6/100], Loss: 0.0591,    Training Accuracy: 97.88%, Validation Accuracy: 96.91%,    Time: 28.175771514574688
Epoch [7/100], Loss: 0.0261,    Training Accuracy: 98.78%, Validation Accuracy: 97.62%,    Time: 28.2382561479296
Epoch [8/100], Loss: 0.2199,    Training Accuracy: 98.63%, Validation Accuracy: 97.27%,    Time: 28.312620908021927
Epoch [9/100], Loss: 0.0158,    Training Accuracy: 98.74%, Validation Accura

Epoch [72/100], Loss: 0.0000,    Training Accuracy: 99.71%, Validation Accuracy: 97.81%,    Time: 30.150720165835487
Epoch [73/100], Loss: 0.0000,    Training Accuracy: 99.68%, Validation Accuracy: 97.93%,    Time: 30.19710775270854
Epoch [74/100], Loss: 0.2013,    Training Accuracy: 99.79%, Validation Accuracy: 97.73%,    Time: 30.228074431419373
Epoch [75/100], Loss: 0.0000,    Training Accuracy: 99.82%, Validation Accuracy: 98.14%,    Time: 30.274377683003742
Epoch [76/100], Loss: 0.0013,    Training Accuracy: 99.57%, Validation Accuracy: 97.64%,    Time: 30.3002402594215
Epoch [77/100], Loss: 0.0012,    Training Accuracy: 99.76%, Validation Accuracy: 97.97%,    Time: 30.317911538210783
Epoch [78/100], Loss: 0.0000,    Training Accuracy: 99.87%, Validation Accuracy: 97.90%,    Time: 30.334050704271366
Epoch [79/100], Loss: 0.0000,    Training Accuracy: 99.52%, Validation Accuracy: 97.53%,    Time: 30.343671901316583
Epoch [80/100], Loss: 0.0099,    Training Accuracy: 99.91%, Validat

In [124]:
# saving model weights

torch.save(model.state_dict(), 'models/abs_weights_784_784_10_100epo__11images_weights_run1.pth')

### porting weights into a simple network just as a sanity check

In [125]:
Two_layres_weights = torch.load('models/abs_weights_784_784_10_100epo__11images_weights_run1.pth')

In [126]:
testmode_Two_layres = SimpleNN(784, 784, 10)

In [127]:
testmode_Two_layres.linear1.weight.data = Two_layres_weights['linear1.weight']
testmode_Two_layres.linear2.weight.data = Two_layres_weights['linear2.weight']
# testmodel_Linear_class.linear1.weight.data = Linear_class_weights['linear1.weight']

In [128]:
calculate_accuracy(loaderforvalid, testmode_Two_layres)

98.11

In [129]:
Power_first = (torch.mean(Two_layres_weights['linear1.weight']**2))

In [130]:
Power_second = (torch.mean(Two_layres_weights['linear2.weight']**2))

In [131]:
Power_first

tensor(0.0228)

In [132]:
Power_second

tensor(0.0240)

### Porting and testing for different alphas

In [139]:
# alphas = [1]

FSDs = torch.sqrt(torch.logspace(-3, 2, 512))


inputsize = 784

hiddensize = 784

outputsize = 10

cleanaccuracies = []
noisyaccuracies = []


In [140]:
start = time.time()

for FSD in FSDs:
    
    my_network_shotnoise = SecureNN2layernoab_homodyne(inputsize, hiddensize, outputsize, noise=True, FSD1=FSD/torch.sqrt(Power_first),FSD2=FSD/torch.sqrt(Power_second))
    my_network_shotnoise.secure1.W.data = Two_layres_weights['linear1.weight']
    my_network_shotnoise.secure2.W.data = Two_layres_weights['linear2.weight']
    acc_1 = calculate_accuracy(loaderforvalid, my_network_shotnoise, secure=False)
    noisyaccuracies.append(acc_1)
    
    print(f"noisy accuracy = {noisyaccuracies[-1]},\
    time = {time.time()-start}")

    start = time.time()


noisy accuracy = 9.98,    time = 6.226252555847168
noisy accuracy = 10.47,    time = 5.631081581115723
noisy accuracy = 9.83,    time = 6.177365064620972
noisy accuracy = 10.39,    time = 5.869917392730713
noisy accuracy = 11.08,    time = 5.952818870544434
noisy accuracy = 9.98,    time = 6.179582595825195
noisy accuracy = 10.08,    time = 5.754180908203125
noisy accuracy = 10.13,    time = 6.234773635864258
noisy accuracy = 10.0,    time = 6.268497943878174
noisy accuracy = 9.96,    time = 5.96399998664856
noisy accuracy = 10.29,    time = 6.354332208633423
noisy accuracy = 10.8,    time = 6.110792875289917
noisy accuracy = 10.09,    time = 6.229742765426636
noisy accuracy = 10.03,    time = 6.21112847328186
noisy accuracy = 10.24,    time = 5.903710603713989
noisy accuracy = 10.03,    time = 6.3602869510650635
noisy accuracy = 10.03,    time = 6.1128621101379395
noisy accuracy = 9.94,    time = 6.073692083358765
noisy accuracy = 10.13,    time = 6.450728416442871
noisy accuracy = 10

noisy accuracy = 15.96,    time = 6.289560794830322
noisy accuracy = 15.79,    time = 6.2605979442596436
noisy accuracy = 15.77,    time = 6.356252431869507
noisy accuracy = 16.86,    time = 6.502885818481445
noisy accuracy = 16.29,    time = 6.300763368606567
noisy accuracy = 17.26,    time = 6.1472392082214355
noisy accuracy = 16.56,    time = 6.396320104598999
noisy accuracy = 17.0,    time = 6.0181920528411865
noisy accuracy = 16.81,    time = 6.275110244750977
noisy accuracy = 18.07,    time = 6.528034925460815
noisy accuracy = 18.13,    time = 6.203021049499512
noisy accuracy = 17.8,    time = 6.389810085296631
noisy accuracy = 17.24,    time = 6.2449891567230225
noisy accuracy = 18.35,    time = 6.320424556732178
noisy accuracy = 18.52,    time = 6.33927321434021
noisy accuracy = 19.48,    time = 6.193951368331909
noisy accuracy = 19.41,    time = 6.389721632003784
noisy accuracy = 18.92,    time = 6.2003014087677
noisy accuracy = 20.11,    time = 6.236707448959351
noisy accurac

noisy accuracy = 92.8,    time = 6.106133222579956
noisy accuracy = 92.49,    time = 6.315290927886963
noisy accuracy = 92.42,    time = 6.181455850601196
noisy accuracy = 93.25,    time = 5.95275616645813
noisy accuracy = 93.41,    time = 5.707200765609741
noisy accuracy = 93.1,    time = 5.565438508987427
noisy accuracy = 93.93,    time = 5.510271072387695
noisy accuracy = 93.7,    time = 5.8647730350494385
noisy accuracy = 93.75,    time = 6.186880350112915
noisy accuracy = 93.97,    time = 5.91178560256958
noisy accuracy = 94.22,    time = 5.815093755722046
noisy accuracy = 94.12,    time = 6.242473125457764
noisy accuracy = 94.12,    time = 5.731858253479004
noisy accuracy = 94.48,    time = 5.6033241748809814
noisy accuracy = 94.5,    time = 6.1122612953186035
noisy accuracy = 94.73,    time = 5.807931661605835
noisy accuracy = 94.86,    time = 6.251118898391724
noisy accuracy = 94.8,    time = 6.412230730056763
noisy accuracy = 94.91,    time = 6.120533227920532
noisy accuracy =

noisy accuracy = 98.0,    time = 6.182231664657593
noisy accuracy = 97.91,    time = 6.368600130081177
noisy accuracy = 98.03,    time = 6.247614860534668
noisy accuracy = 97.98,    time = 5.205843687057495
noisy accuracy = 97.98,    time = 5.956197023391724
noisy accuracy = 98.03,    time = 6.236898183822632
noisy accuracy = 97.95,    time = 6.324038505554199
noisy accuracy = 98.08,    time = 6.413276433944702
noisy accuracy = 97.92,    time = 6.090365886688232
noisy accuracy = 97.97,    time = 6.4439051151275635
noisy accuracy = 98.02,    time = 6.242887735366821
noisy accuracy = 98.12,    time = 6.386364698410034
noisy accuracy = 98.09,    time = 6.46554708480835
noisy accuracy = 98.0,    time = 6.263397932052612
noisy accuracy = 97.94,    time = 6.294710397720337
noisy accuracy = 98.05,    time = 6.358391523361206
noisy accuracy = 98.01,    time = 6.3075852394104
noisy accuracy = 98.08,    time = 6.345686435699463
noisy accuracy = 98.08,    time = 6.193160533905029
noisy accuracy =

In [59]:
start = time.time()

for alpha in alphas:
    
    my_network_shotnoise = SecureNN2layer_2sum(inputsize, hiddensize, outputsize, noise=True, alpha1=alpha/torch.sqrt(Two_layres_power_first),alpha2=alpha/torch.sqrt(Two_layres_power_second))
    
    my_network_shotnoise.secure1.secure1.W.data = Two_layres_weights['linear1.weight']
    my_network_shotnoise.secure1.secure2.W.data = Two_layres_weights['linear2.weight']
    my_network_shotnoise.secure2.secure1.W.data = Two_layres_weights['linear1.weight']
    my_network_shotnoise.secure2.secure2.W.data = Two_layres_weights['linear2.weight']
    
    acc_1 = calculate_accuracy(loaderforvalid, my_network_shotnoise, secure=False)
    noisyaccuracies_perm.append(acc_1)
    
    print(f"noisy accuracy = {noisyaccuracies_perm[-1]},\
    time = {time.time()-start}")

    start = time.time()


noisy accuracy = 12.41,    time = 2.4308245182037354
noisy accuracy = 13.43,    time = 2.684985637664795
noisy accuracy = 12.89,    time = 2.453415632247925
noisy accuracy = 13.53,    time = 2.534616231918335
noisy accuracy = 12.89,    time = 2.5739102363586426
noisy accuracy = 13.32,    time = 2.592500925064087
noisy accuracy = 13.4,    time = 2.5785915851593018
noisy accuracy = 14.38,    time = 2.472114324569702
noisy accuracy = 13.36,    time = 2.471698760986328
noisy accuracy = 13.97,    time = 2.605441093444824
noisy accuracy = 13.99,    time = 2.5377211570739746
noisy accuracy = 14.02,    time = 2.461756706237793
noisy accuracy = 13.96,    time = 2.5778939723968506
noisy accuracy = 14.03,    time = 2.4702072143554688
noisy accuracy = 14.41,    time = 2.53932523727417
noisy accuracy = 14.62,    time = 2.4657108783721924
noisy accuracy = 15.88,    time = 2.5172367095947266
noisy accuracy = 15.73,    time = 2.529005527496338
noisy accuracy = 15.64,    time = 2.651350736618042
noisy 

noisy accuracy = 97.75,    time = 2.3697564601898193
noisy accuracy = 97.78,    time = 2.5383238792419434
noisy accuracy = 97.69,    time = 2.549370288848877
noisy accuracy = 97.78,    time = 2.570044755935669
noisy accuracy = 97.79,    time = 2.5263969898223877
noisy accuracy = 97.77,    time = 2.531607151031494
noisy accuracy = 97.62,    time = 2.501436233520508
noisy accuracy = 97.7,    time = 2.5574960708618164
noisy accuracy = 97.78,    time = 2.633883476257324
noisy accuracy = 97.78,    time = 3.1790802478790283
noisy accuracy = 97.75,    time = 2.7638630867004395
noisy accuracy = 97.76,    time = 2.5918214321136475
noisy accuracy = 97.7,    time = 2.6944501399993896
noisy accuracy = 97.71,    time = 2.6322388648986816
noisy accuracy = 97.79,    time = 2.703538417816162
noisy accuracy = 97.81,    time = 2.7023491859436035
noisy accuracy = 97.81,    time = 2.862415075302124
noisy accuracy = 97.69,    time = 2.5430843830108643
noisy accuracy = 97.68,    time = 2.610766887664795
noi

In [98]:
for alpha in alphas_linear:
    my_network_shotnoise_Linear_class = SecureNN1layernoab_homodyne(inputsize, outputsize, noise=True, alpha=alpha)
    my_network_shotnoise_Linear_class.secure1.W.data = Linear_class_weights['linear1.weight']
    acc_linear = calculate_accuracy(loaderforvalid, my_network_shotnoise_Linear_class, secure=False)
    noisyaccuracies_Linear.append(acc_linear)
    
    print(f"noisy accuracy = {noisyaccuracies_Linear[-1]},\
    time = {time.time()-start}")


noisy accuracy = 13.18,    time = 601.8503766059875
noisy accuracy = 13.24,    time = 604.1128273010254
noisy accuracy = 13.71,    time = 606.360936164856
noisy accuracy = 15.81,    time = 608.5634233951569
noisy accuracy = 15.9,    time = 610.8291156291962
noisy accuracy = 18.36,    time = 613.0806677341461
noisy accuracy = 20.73,    time = 615.50532746315
noisy accuracy = 24.11,    time = 617.7904245853424
noisy accuracy = 28.04,    time = 620.2076599597931
noisy accuracy = 33.01,    time = 622.4919300079346
noisy accuracy = 38.63,    time = 624.8300333023071
noisy accuracy = 45.89,    time = 627.0925409793854
noisy accuracy = 52.37,    time = 629.3908998966217
noisy accuracy = 60.55,    time = 631.6266794204712
noisy accuracy = 67.43,    time = 633.9396030902863
noisy accuracy = 73.57,    time = 636.2220795154572
noisy accuracy = 78.28,    time = 638.5775594711304
noisy accuracy = 82.07,    time = 640.7650957107544
noisy accuracy = 84.51,    time = 643.0469388961792
noisy accuracy =

In [141]:
print(noisyaccuracies)

[9.98, 10.47, 9.83, 10.39, 11.08, 9.98, 10.08, 10.13, 10.0, 9.96, 10.29, 10.8, 10.09, 10.03, 10.24, 10.03, 10.03, 9.94, 10.13, 10.27, 10.33, 9.65, 10.46, 10.4, 10.26, 10.23, 10.7, 10.01, 10.09, 10.4, 11.17, 10.16, 10.0, 10.72, 10.2, 10.1, 9.58, 9.56, 10.19, 10.65, 10.27, 10.88, 10.26, 10.56, 10.41, 10.52, 10.34, 10.83, 10.04, 10.77, 10.42, 10.93, 10.27, 10.68, 10.6, 10.89, 10.77, 9.96, 10.47, 10.44, 10.93, 10.81, 11.04, 10.77, 10.73, 10.48, 10.72, 10.68, 10.13, 10.84, 10.3, 10.47, 10.99, 10.57, 10.94, 10.84, 10.63, 10.94, 11.14, 11.1, 10.94, 10.85, 10.82, 11.45, 11.4, 10.74, 10.66, 10.94, 10.95, 10.87, 11.61, 11.46, 10.96, 11.61, 10.66, 10.49, 10.69, 11.72, 11.84, 11.57, 11.06, 11.68, 11.72, 11.31, 11.55, 11.67, 11.71, 11.72, 11.73, 11.98, 11.89, 11.61, 12.1, 11.51, 12.35, 12.4, 12.16, 12.32, 12.35, 12.28, 12.47, 12.39, 11.96, 12.54, 12.85, 12.88, 12.54, 12.69, 12.93, 12.91, 12.69, 12.78, 12.85, 13.29, 13.27, 13.5, 13.81, 13.54, 13.48, 13.55, 13.83, 14.09, 13.7, 14.28, 13.6, 14.21, 14.

In [60]:
print(noisyaccuracies_perm)

[12.41, 13.43, 12.89, 13.53, 12.89, 13.32, 13.4, 14.38, 13.36, 13.97, 13.99, 14.02, 13.96, 14.03, 14.41, 14.62, 15.88, 15.73, 15.64, 16.26, 16.65, 16.36, 16.19, 16.8, 16.82, 17.6, 17.56, 17.56, 18.49, 18.43, 19.29, 19.4, 19.74, 20.77, 20.45, 20.89, 20.84, 21.89, 22.35, 23.22, 22.77, 23.45, 25.16, 25.14, 27.57, 25.82, 27.22, 28.03, 29.09, 30.07, 31.28, 32.1, 31.47, 33.71, 33.76, 34.52, 36.32, 37.61, 38.32, 39.99, 41.59, 42.06, 42.71, 44.64, 45.72, 47.21, 48.49, 49.57, 50.47, 52.04, 54.46, 55.3, 55.87, 58.33, 59.89, 61.42, 63.03, 64.54, 65.27, 66.65, 67.94, 69.57, 70.71, 72.31, 73.71, 74.66, 76.32, 77.83, 79.05, 79.74, 81.18, 81.99, 83.22, 84.05, 85.03, 85.92, 87.01, 87.42, 88.42, 89.3, 89.67, 90.42, 90.92, 91.37, 91.7, 92.08, 92.56, 93.24, 93.41, 93.81, 94.4, 94.34, 94.62, 95.13, 95.31, 95.18, 95.58, 95.84, 95.98, 96.1, 96.1, 96.25, 96.56, 96.64, 96.72, 96.52, 96.8, 96.79, 96.73, 96.91, 96.95, 97.11, 97.12, 97.16, 97.17, 97.2, 97.25, 97.3, 97.2, 97.4, 97.31, 97.37, 97.4, 97.42, 97.36, 9